# Import amd Setting

In [1]:
import sys, os
import numpy as np
import pandas as pd

sys.path.append(os.path.join('..'))
import original_lingam

sys.path.append(os.path.join('..'))
import BF_original

sys.path.append(os.path.join('..'))
import proposed_lingam

sys.path.append(os.path.join('..'))
import false_lingam

sys.path.append(os.path.join('..', 'sythetic_data'))
import generate
from generate import generate

sys.path.append(os.path.join('..', 'related_methods'))
import notears

sys.path.append(os.path.join('..', 'related_methods'))
import ges

from supplement import criteria
from supplement import threshold

import random
import scipy
import networkx
import seaborn as sns
import matplotlib.pyplot as plt
# import sempler_tool
import supplement

np.set_printoptions(precision=3, suppress=True)
# np.random.seed(2022)

#  Experiment (Synthetic) & Visualize

## 問題設定
### ランダムに変数の生成機構 (線形の構造方程式) を設定し、そこから生成されたデータをもとに各因果探索手法により生成機構を推定する。

In [ ]:
x = [3, 4, 5, 6, 7, 8, 9, 10]
num_exp = 5
n = 300

r = []
for _ in range(num_exp):
    r.append(random.randint(0, 100))
    
methods = ["Original LiNGAM","NO TEARS", "GES"]

original_LiNGAM_result = [[0] * num_exp for _ in range(len(x))]
notears_result = [[0] * num_exp for _ in range(len(x))]
ges_result = [[0] * num_exp for _ in range(len(x))]

results_mean = [[0] * len(x) for i in range(len(methods))]
results_sd = [[0] * len(x) for i in range(len(methods))]

# Original Direct LiNGAM
for i in range(len(x)):
    for j in range(num_exp):
        X = generate(n, x[i], r[j])
        model_original_LiNGAM = original_lingam.DirectLiNGAM("kernel")
        model_original_LiNGAM.fit(X[0])
        acyclic_W = threshold.threshold_output(model_original_LiNGAM.adjacency_matrix_)
        correct = threshold.threshold_output(X[1])
        original_LiNGAM_result[i][j] = criteria.SHD(acyclic_W, correct)
    results_mean[0][i] = np.mean(original_LiNGAM_result[i])
    results_sd[0][i] = np.std(original_LiNGAM_result[i])

# NO TEARS
for i in range(len(x)):
    for j in range(num_exp):
        X = generate(n, x[i], r[j])
        output_dict = notears.run(notears.notears_standard, X[0], notears.loss.least_squares_loss, notears.loss.least_squares_loss_grad, e=1e-8, verbose=False)
        acyclic_W = threshold.threshold_output(output_dict[0]['W'])
        correct = threshold.threshold_output(X[1])
        notears_result[i][j] = criteria.SHD(acyclic_W, correct)
    results_mean[1][i] = np.mean(notears_result[i])
    results_sd[1][i] = np.std(notears_result[i])

    
# GES
for i in range(len(x)):
    for j in range(num_exp):
        X = generate(n, x[i], r[j])
        estimate, score = ges.fit_bic(X[0])
        correct = threshold.threshold_output(X[1])
        ges_result[i][j] = criteria.SHD(estimate, correct)
    results_mean[2][i] = np.mean(ges_result[i])
    results_sd[2][i] = np.std(ges_result[i])



plt.style.use('default')
sns.set()
sns.set_style('whitegrid')
sns.set_palette('Set1')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
x = np.array(x)

for i in range(len(methods)):
    ax.errorbar(x, results_mean[i], yerr=results_sd[i], marker='o', label=methods[i], capthick=1, capsize=8, lw=1)

# ax.set_ylim(0, 20)
ax.set_xlabel("The number of variables")
ax.set_ylabel('SHD')

ax.legend()
plt.show()

Ground Truth:
[[ 0 -3  1]
 [ 0  0 -2]
 [ 0  0  0]]
Ground Truth:
[[ 0 -3 -2]
 [ 0  0  1]
 [ 0  0  0]]
Ground Truth:
[[0 0 1]
 [0 0 0]
 [0 0 0]]
Ground Truth:
[[0 2 0]
 [0 0 0]
 [0 0 0]]
Ground Truth:
[[ 0  1 -3]
 [ 0  0  1]
 [ 0  0  0]]
Ground Truth:
[[ 0 -2  2 -1]
 [ 0  0 -3 -3]
 [ 0  0  0 -2]
 [ 0  0  0  0]]
Ground Truth:
[[ 0 -1 -1 -1]
 [ 0  0  0  2]
 [ 0  0  0  2]
 [ 0  0  0  0]]
Ground Truth:
[[ 0  2 -3  2]
 [ 0  0 -1  0]
 [ 0  0  0 -2]
 [ 0  0  0  0]]
Ground Truth:
[[ 0 -3  2 -3]
 [ 0  0  2 -1]
 [ 0  0  0  1]
 [ 0  0  0  0]]
Ground Truth:
[[ 0 -3  1  0]
 [ 0  0  1 -3]
 [ 0  0  0  0]
 [ 0  0  0  0]]
Ground Truth:
[[ 0  0  2 -3 -3]
 [ 0  0 -2 -1  1]
 [ 0  0  0 -3  0]
 [ 0  0  0  0 -3]
 [ 0  0  0  0  0]]
Ground Truth:
[[ 0  1 -1 -1 -3]
 [ 0  0  1 -2  0]
 [ 0  0  0  0  2]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
Ground Truth:
[[ 0 -2 -2  2 -3]
 [ 0  0 -3  1 -1]
 [ 0  0  0 -3 -3]
 [ 0  0  0  0  2]
 [ 0  0  0  0  0]]
Ground Truth:
[[ 0  1 -2 -2  1]
 [ 0  0  2  2  2]
 [ 0  0  0  0 -3]
 [ 0 

Ground Truth:
[[ 0  1 -1  1  0  0]
 [ 0  0 -3  0 -1  2]
 [ 0  0  0 -3 -2  2]
 [ 0  0  0  0 -3  2]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
Ground Truth:
[[ 0 -2 -1  1  0  1]
 [ 0  0  2 -1  2  0]
 [ 0  0  0 -3  0 -3]
 [ 0  0  0  0 -2  1]
 [ 0  0  0  0  0 -1]
 [ 0  0  0  0  0  0]]
Ground Truth:
[[ 0  1  2  1 -1 -2]
 [ 0  0  0  0  1 -3]
 [ 0  0  0 -3 -3  0]
 [ 0  0  0  0  1  0]
 [ 0  0  0  0  0  2]
 [ 0  0  0  0  0  0]]
Ground Truth:
[[ 0  0 -3  1 -2  0]
 [ 0  0 -1  2 -2 -1]
 [ 0  0  0  1 -3 -2]
 [ 0  0  0  0  2 -3]
 [ 0  0  0  0  0  2]
 [ 0  0  0  0  0  0]]
Ground Truth:
[[ 0 -3  0 -2  2  2]
 [ 0  0 -3 -3 -3  1]
 [ 0  0  0 -2 -1  1]
 [ 0  0  0  0  1  0]
 [ 0  0  0  0  0 -1]
 [ 0  0  0  0  0  0]]
Ground Truth:
[[ 0 -2 -2  1  1 -2  1]
 [ 0  0 -3  0 -2  2  0]
 [ 0  0  0 -3  0  1 -1]
 [ 0  0  0  0  2 -2  2]
 [ 0  0  0  0  0  1  2]
 [ 0  0  0  0  0  0 -3]
 [ 0  0  0  0  0  0  0]]
Ground Truth:
[[ 0  1  2  0 -1 -1 -1]
 [ 0  0 -1  2 -3  1 -3]
 [ 0  0  0  0  1  2  2]
 [ 0  0  0  0  0  0 -2]
 [

# Experiment (Real) & Visualize

## 問題設定
### 因果関係に関する正解データを持つリアルワールドデータセットをもとに因果関係を推定する。

In [ ]:
df = pd.read_csv("Sachs_dataset.csv")
df = df[:1500]
correct = [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], 
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1], [1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1], 
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
    
methods = ["Original LiNGAM", "NO TEARS", "GES"]

# Original Direct LiNGAM
model_original_LiNGAM = original_lingam.DirectLiNGAM("kernel")
model_original_LiNGAM.fit(df)
acyclic_W = threshold.threshold_output(model_original_LiNGAM.adjacency_matrix_)
print("Original Direct LiNGAM")
print(criteria.SHD(acyclic_W, correct))

# # True Proposed Direct LiNGAM
# model_proposed_LiNGAM = proposed_lingam.DirectLiNGAM("kernel")
# model_proposed_LiNGAM.fit(df)
# acyclic_W = threshold.threshold_output(model_proposed_LiNGAM.adjacency_matrix_)
# print("True Proposed Direct LiNGAM")
# print(criteria.SHD(acyclic_W, correct))

# # False Proposed Direct LiNGAM
# model_false_LiNGAM = false_lingam.DirectLiNGAM("kernel")
# model_false_LiNGAM.fit(df)
# acyclic_W = threshold.threshold_output(model_false_LiNGAM.adjacency_matrix_)
# print("False Proposed Direct LiNGAM")
# print(criteria.SHD(acyclic_W, correct))

# NO TEARS
output_dict = notears.run(notears.notears_standard, df, notears.loss.least_squares_loss, notears.loss.least_squares_loss_grad, e=1e-8, verbose=False)
acyclic_W = threshold.threshold_output(output_dict[0]['W'])
print("NO TEARS")
print(criteria.SHD(acyclic_W, correct))

# GES
estimate, score = ges.fit_bic(df)
print("GES")
print(criteria.SHD(estimate, correct))

